<a href="https://colab.research.google.com/github/Tarekhelmy/NobianHachathon/blob/main/predictive_maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1704]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1705]:
# Import libraries 
import numpy as np
import pandas as pd
from time import time
from os import path
import random
from copy import deepcopy
import tensorflow as tf

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, sampler
from torchvision import transforms
from torchsummary import summary

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from scipy import signal

def fastfourier(Signal,dt):
    Time_freq = np.fft.rfftfreq(len(Signal),dt)
    num_freq = np.fft.rfft(Signal)
    return Time_freq, num_freq

def arrayfourier(array,dt):
    result = []
    for signal in array:
      [temp, y] = fastfourier(signal,dt)
      result.append(y)
    return np.array(result)

dt = 201.47284714485522

In [1706]:
#Import Clogged Dataframe
clog_df = pd.read_csv('/content/drive/MyDrive/Predictive Maintanence Hackathon/clogged.csv')
clog_df = clog_df.drop('Unnamed: 0', axis=1)
clog_df.insert(0, "clog", 1)
clog_df.head()

,clog,0,1,2,3,4,5,6,7,8,...,340,341,342,343,344,345,346,347,348,349
0,1,0.666004,0.826448,0.915121,0.810420,0.291874,0.633545,0.964287,1.310707,1.452484,...,22.595003,21.210184,22.227953,20.938693,21.863215,20.664924,21.500721,20.388945,21.140401,20.110826
1,1,-1.026032,-0.997256,-0.811005,1.008633,0.923112,0.407431,0.305532,-0.194155,-0.092365,...,4.406733,4.395920,4.897854,0.657127,6.923686,7.927258,6.280515,7.958984,6.134981,4.169182
2,1,-0.273825,-0.196117,-0.256358,-0.246636,-0.164783,-0.137436,-0.048262,0.213430,-0.030943,...,0.256950,0.517147,0.851276,1.112187,0.761099,-0.206930,-0.163962,0.106823,0.409989,1.433046
3,1,3.467870,4.462160,3.023227,2.853359,3.013184,2.952436,2.520755,2.798162,3.311579,...,25.589749,25.875554,25.511689,25.837068,25.536279,25.800436,25.559039,25.765610,3.372606,-4.046035
4,1,0.204302,0.147773,0.112367,0.114957,0.080994,0.082176,0.049588,0.049427,0.018149,...,1.173801,1.412043,2.293380,-2.140201,-1.872519,-1.378186,0.188232,1.110314,1.517383,-1.073308


In [1707]:
#Import Unclogged Dataframe
unclog_df = pd.read_csv('/content/drive/MyDrive/Predictive Maintanence Hackathon/unclogged.csv')
unclog_df = unclog_df.drop('Unnamed: 0', axis=1)
unclog_df.insert(0, "clog", 0)
unclog_df.head()

,clog,0,1,2,3,4,5,6,7,8,...,340,341,342,343,344,345,346,347,348,349
0,0,0.288239,0.256936,0.072581,-0.146370,0.179817,0.725468,0.830655,0.486467,0.069150,...,0.223321,0.186109,0.124190,0.085354,0.025000,0.116077,0.226043,0.345827,0.456856,0.575622
1,0,0.889510,0.914475,0.806630,0.878618,0.942275,1.128650,1.186099,1.112862,0.904497,...,0.638855,14.946922,-7.206893,-7.138776,5.158324,4.278329,3.212653,-1.357792,1.423712,-2.840907
2,0,0.313277,0.399624,0.444959,0.406850,0.289053,0.214125,0.098005,0.323226,1.103166,...,-0.362814,-0.342052,-0.487057,-0.710168,-0.623719,-0.542732,-0.450282,-0.369476,-0.301635,-0.296763
3,0,4.554387,4.746501,4.263385,4.447203,4.323678,4.562328,3.962739,3.178653,2.866642,...,3.358185,2.769172,2.320687,3.731556,3.481709,3.022022,3.407570,3.586918,2.214383,2.811060
4,0,0.449453,0.515530,0.570562,0.690228,0.747720,0.703795,0.653020,0.673568,0.625636,...,0.577302,0.542462,0.519560,0.484810,0.559392,0.769051,0.993651,1.203539,1.089710,0.648543


In [1716]:
# MORE DATA HANDLING KILL ME 
df = pd.concat([clog_df, unclog_df])
target_column = ['clog'] 
input_dim = 250
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()

X = df[predictors].values[:,:input_dim]
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(X_train.shape); print(X_test.shape)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

count_classes = y_test.shape[1]
print(count_classes)

(76, 250)
(20, 250)
2


In [1717]:
# MOREEEEEEEEEEEEE
# df = pd.concat([clog_df, unclog_df])
# target_column = ['clog'] 
# predictors = list(set(list(df.columns))-set(target_column))
# df[predictors] = df[predictors]/df[predictors].max()

# X = df[predictors].values[:,:300]
# y = df[target_column].values

# avg = np.mean(X, axis=1)
# diff = X-avg.reshape(96,1)
# X = np.cumsum(diff, axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
# print(X_train.shape); print(X_test.shape)
# input_dim = X.shape[1] 

# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# count_classes = y_test.shape[1]
# print(count_classes)

In [1718]:
# MORE DATA HANDLING 2 ELECTRIC BOOGALOO
# df = pd.concat([clog_df, unclog_df])
# target_column = ['clog'] 
# predictors = list(set(list(df.columns))-set(target_column))
# df[predictors] = df[predictors]/df[predictors].max()

# dt = 201.47284714485522
# X = arrayfourier(df[predictors].values, dt)
# y = df[target_column].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=69)
# print(X_train.shape); print(X_test.shape)

# input_dim = X_train.shape[1]

# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# count_classes = y_test.shape[1]
# print(count_classes)

In [1719]:
# ANN
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, activation='relu', input_dim=input_dim))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
print(model.summary())

Model: "sequential_163"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_515 (Dense)           (None, 100)               25100     
                                                                 
 dropout_89 (Dropout)        (None, 100)               0         
                                                                 
 dense_516 (Dense)           (None, 200)               20200     
                                                                 
 dense_517 (Dense)           (None, 50)                10050     
                                                                 
 dense_518 (Dense)           (None, 2)                 102       
                                                                 
Total params: 55,452
Trainable params: 55,452
Non-trainable params: 0
_________________________________________________________________
None


In [1720]:
# Fully Convolutional Network
# model = tf.keras.Sequential()
# model.add(tf.keras.Input(shape=(input_dim,1)))
# model.add(tf.keras.layers.Conv1D(128, 8, activation='relu',use_bias=True))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.ReLU())


# model.add(tf.keras.layers.Conv1D(256, 5, activation='relu',use_bias=True))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.ReLU())


# model.add(tf.keras.layers.Conv1D(128, 3, activation='relu',use_bias=True))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.ReLU())

# model.add(tf.keras.layers.GlobalAveragePooling1D())

# model.add(tf.keras.layers.Dense(2, activation='softmax'))
# print(model.summary())

In [1721]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.TruePositives(), tf.keras.metrics.FalsePositives()])

In [1722]:
# build the model
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
3/3 [==============================] - 1s 6ms/step - loss: 0.6939 - accuracy: 0.5395 - true_positives_72: 41.0000 - false_positives_70: 35.0000
Epoch 2/20
3/3 [==============================] - 0s 5ms/step - loss: 0.7078 - accuracy: 0.5263 - true_positives_72: 40.0000 - false_positives_70: 36.0000
Epoch 3/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6696 - accuracy: 0.6184 - true_positives_72: 47.0000 - false_positives_70: 29.0000
Epoch 4/20
3/3 [==============================] - 0s 6ms/step - loss: 0.6570 - accuracy: 0.6053 - true_positives_72: 46.0000 - false_positives_70: 30.0000
Epoch 5/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6604 - accuracy: 0.5789 - true_positives_72: 44.0000 - false_positives_70: 32.0000
Epoch 6/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6387 - accuracy: 0.6579 - true_positives_72: 50.0000 - false_positives_70: 26.0000
Epoch 7/20
3/3 [==============================] - 0s 5ms/step - loss: 

In [1723]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1]*100, 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1]*100, 1 - scores2[1])) 
print(scores2[2:])
print('Clog Prediction before:'+str((350-input_dim)*dt/3600))

Accuracy on training data: 85.52631735801697% 
 Error on training data: 0.14473682641983032
Accuracy on test data: 64.99999761581421% 
 Error on test data: 0.3500000238418579
[13.0, 7.0]
Clog Prediction before:5.596467976245979
